Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [198]:
from random import choices, random, randint
import numpy as np
from copy import deepcopy
from IPython.display import clear_output

import lab9_lib

# GOAL: maximize fitness, minimize calls

In [199]:
fitness = lab9_lib.make_problem(5)
for n in range(20):
    ind = choices([0, 1], k=100)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

1011100100101011101000110001111110000100101010010001001001000110001000010111010010010001010100010110: 13.12%
1010001001000011000110101011011111001010001110000100010000010110101001100010100111110101110111100101: 13.90%
1101001100010000010101111110000011010111001000000000010101100001110010110000001000101111000001010101: 19.12%
0110110100011100000011001101001010110101010011100101011010010011101000000101111000010011110001001101: 20.91%
1100110010000011100001001000011001010100011000101001101000110101100101011001001100101011001110000001: 9.91%
1111101111001100100101101001101000110010110100100010011101001010010101011000100100000000000110011001: 19.00%
0111110101010000001110011001111110101011111011001110001011011011010010001100100101110001011011000110: 11.67%
1111010010001001111111101111101100101011101101100111010000011110011110110000111000111011001011000100: 37.83%
0000110110111010111101010101100010101101000011101110000101101000000011000101101010011001111010001000: 9.89%
000111000100011111110

Our code below

In [200]:
POPULATION_SIZE = 500
OFFSPRING_SIZE = 200
LOCI = 1000
BIT_FLIP_PROBABILITY = 0.15
# SWAP_PROBABILITY = 150 * 1/LOCI
SWAP_PROBABILITY = 0.5
NUM_GENERATION = 20


class Individual:
    def __init__(self):
        self.genotype = choices([0, 1], k=LOCI)
        self.fitness = float("-inf")

In [201]:
# Mutation / recombination or both
def parent_selection(
    population: list[Individual], tournament_size: int
) -> Individual:
    # we also want to take the last best one.
    parents_idx = np.random.choice(
        range(len(population)), size=tournament_size, replace=False
    )
    parents = [population[idx] for idx in parents_idx]
    return max(parents, key=lambda i: i.fitness)


def uniform_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    p1 = ind1.fitness / (ind1.fitness + ind2.fitness)
    mask = np.random.choice([True, False], size=LOCI, p=[p1, 1 - p1])
    gene = np.where(mask, ind1.genotype, ind2.genotype)
    new_ind = Individual()
    new_ind.genotype = gene.tolist()
    return new_ind

'''NOT USEFUL'''
# def uniform_cut_xover_old(ind1: Individual, ind2: Individual) -> list[Individual]:
#     ind1.genotype = np.array(ind1.genotype)
#     ind2.genotype = np.array(ind2.genotype)
#     swap_mask = np.random.rand(len(ind1.genotype)) < SWAP_PROBABILITY
#     temp = np.copy(ind1.genotype[swap_mask])
#     ind1.genotype[swap_mask] = ind2.genotype[swap_mask]
#     ind2.genotype[swap_mask] = temp
#     return [ind1, ind2]


def mutate(parent: Individual) -> Individual:
    new_offspring = deepcopy(parent)
    for i in range(LOCI):
        if random() < BIT_FLIP_PROBABILITY:
            new_offspring.genotype[i] = int(not new_offspring.genotype[i])
    return new_offspring


def offspring_generation(
    parent1: Individual, parent2: Individual, mutation_probability: int
) -> Individual:
    offspring = uniform_cut_xover(parent1, parent2)
    offspring = mutate(offspring) if random() < random() < mutation_probability else offspring
    offspring.fitness = fitness(offspring.genotype)
    return offspring

# TODO adattare la population size
# TODO adattare mutation rate e tournament size.
# TODO


def ea() -> Individual:
    fitness_list = [0]
    # starting pouplation of POPULATION_SIZE individuals
    population = [Individual() for _ in range(POPULATION_SIZE)]
    for p in population:
        p.fitness = fitness(p.genotype)
    best_fitness = population[0].fitness
    gen = 0
    fitness_stall = 0 
    mutation_probability = 0.2
    tournament_size = 3
    while fitness_stall < 100 or gen < NUM_GENERATION:
        num_of_better_offspring = 0
        for _ in range(OFFSPRING_SIZE):
            parent1 = parent_selection(population, tournament_size)
            parent2 = parent_selection(population, tournament_size)
            offspring = offspring_generation(
                parent1, parent2, mutation_probability
            )
            #population.extend([offspring]) if isinstance(offspring, Individual) else population.extend(offspring)            # tracking the number of offspring better than the best old one.
            population.extend([offspring])

        population.sort(key=lambda i: i.fitness, reverse=True)
        # always keep the first POPULATION_SIZE best individuals
        population = population[:POPULATION_SIZE]

        #  Self-adapting the values.
        # TODO favour mutation insted of recombination if the fitness it's not high (<0.65).
        if fitness_stall > 5:
            # If we are able to generate 5% of offsprings better than the father,
            # We should explore more.
            mutation_probability *= 1.3

        if best_fitness == population[0].fitness:
            fitness_stall += 1
        else:
            best_fitness = population[0].fitness
            fitness_stall = 0
            mutation_probability = 0.2

        # fitness_list.append(best_fitness)
        gen += 1

        clear_output(wait=True)
        print(
            f"gen #{gen}, fitness: {best_fitness}, score: {best_fitness/gen * 1000:.4f}"
        )

    return population[0]

In [204]:
# instance = [1, 2, 5, 10]
instance = [10]
for k in instance:
    fitness = lab9_lib.make_problem(k)
    best_fitness = ea().fitness
    print(f"\nBest individual fitness: {best_fitness}, Fitness calls: {fitness.calls} -> Score2: {best_fitness/fitness.calls*10000000:.4f}")

gen #126, fitness: 0.4143, score: 3.2881

Best individual fitness: 0.4143, Fitness calls: 25700 -> Score2: 161.2062


## Island model

In [ ]:
class Island:
   def __init__(self, population_size, offspring_size, LOCI):
      self.population_size = population_size
      self.population = [Individual() for _ in range(population_size)]
      self

In [203]:
def initialize_populations(num_islands, population_size, chromosome_length):
    populations = []
    for _ in range(num_islands):
        population = generate_random_population(population_size, chromosome_length)
        populations.append(population)
    return populations

def evolve_island(population, num_generations, mutation_rate, crossover_rate):
    for _ in range(num_generations):
        parents = select_parents(population)
        offspring = crossover(parents, crossover_rate)
        offspring = mutate(offspring, mutation_rate)
        evaluate_fitness(offspring)
        population = select_survivors(population, offspring)
    return population

def exchange_individuals(populations, exchange_rate):
    for i in range(len(populations)):
        if random() < exchange_rate:
            other_island = (i + 1) % len(populations)
            individuals_to_exchange = select_individuals_to_exchange(populations[i])
            populations[i].remove(individuals_to_exchange)
            populations[other_island].add(individuals_to_exchange)

def island_model(num_islands, population_size, chromosome_length, num_generations, mutation_rate, crossover_rate, exchange_rate):
    populations = initialize_populations(num_islands, population_size, chromosome_length)

    for _ in range(num_generations):
        for i in range(num_islands):
            populations[i] = evolve_island(populations[i], 1, mutation_rate, crossover_rate)

        exchange_individuals(populations, exchange_rate)

    return populations
